In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
movies =[]
url=['https://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2018&p=.htm'
    ]

for i in range(2): 
    page = requests.get(url[i])
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    tr_elements = tr_elements[9:109]
    mov = []
    for xd in range(len(tr_elements)):
        for a in tr_elements[xd]:      
            try:
                mov.append(int(a.text_content()))
            except:
                mov.append(a.text_content())
                pass
            try:
                url_mov = a.getchildren()[0].getchildren()[0].getchildren()[0].attrib['href'] #XD
                page2 = requests.get('https://www.boxofficemojo.com'+url_mov)
                doc2 = lh.fromstring(page2.content)
                tr_elements2 = doc2.xpath('//tr')
                mov.append(tr_elements2[7].getchildren()[0].getchildren()[0].text_content()) #distributor full name
                mov.append(tr_elements2[7].getchildren()[1].getchildren()[0].text_content()) #release date
                mov.append(tr_elements2[8].getchildren()[0].getchildren()[0].text_content()) # genre
                mov.append(tr_elements2[8].getchildren()[1].getchildren()[0].text_content()) #runtime
                mov.append(tr_elements2[9].getchildren()[0].getchildren()[0].text_content()) #rating
                mov.append(tr_elements2[9].getchildren()[1].getchildren()[0].text_content()) #budget
                if(tr_elements2[15].getchildren()[0].text_content().replace(u'\xa0', u'').replace('+','') == 'Foreign:'):
                    mov.append(tr_elements2[15].getchildren()[1].text_content().replace(u'\xa0', u'')) #rest world
                else:
                    mov.append('no dist')
            except: 
                pass #XD2
        movies.append(mov)
        mov = []

In [3]:
headers =['#','name','distributor_full_name','release_date','genre','runtime','rating','budget','rest_world','studio','total_gross','theaters_total','opening','theaters_opening','open_date','close_date',]
df = pd.DataFrame(movies,columns = headers)
df.to_csv('movies.csv')

In [42]:
df = pd.read_csv('movies.csv')

In [45]:
df['total_gross'].isnull().values.any()

False

In [43]:
#usuwanie $ itp
df['total_gross'].apply(lambda x: str(x))
df['opening'].apply(lambda x: str(x))
df['total_gross'] = df['total_gross'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
df['opening'] = df['opening'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
df['opening'].fillna(df['opening'].mean(),inplace=True)
df.head()

,Unnamed: 0,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date
0,0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,2 hrs. 20 min.,PG-13,NaN,"$646,853,595",BV,700059566.0,"4,084",202003951.0,"4,020",2/16,8/9
1,1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,2 hrs. 36 min.,PG-13,NaN,"$1,369,544,272",BV,678815482.0,"4,474",257698183.0,"4,474",4/27,9/13
2,2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,1 hrs. 58 min.,PG,NaN,"$634,221,293",BV,608581744.0,"4,410",182687905.0,"4,410",6/15,12/13
3,3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,2 hrs. 8 min.,PG-13,$170 million,"$891,764,701",Uni.,417719760.0,"4,485",148024610.0,"4,475",6/22,10/4
4,4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,2 hrs. 19 min.,PG-13,NaN,"$809,600,000",WB,334089785.0,"4,184",67873522.0,"4,125",12/21,-


In [46]:
feats = ['opening']
X = df[ feats ].values
y = df['total_gross'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3) 

X_train.shape, X_test.shape

((140, 1), (60, 1))

In [ ]:
### rekordy posortowane?
### zmienne nie kategoryczne?
### spróbować zwykłej regresji liniowej

In [49]:
# model = DecisionTreeClassifier(max_depth=10)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)

# accuracy_score (y_test, y_pred)

0.0

In [50]:
# model = RandomForestClassifier(max_depth=20,n_estimators=50)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)

# accuracy_score (y_test, y_pred)

0.0